CDExcelMessenger.py

CDExcelMessenger.py is the python module that contains the functions that allows you to pass data between an Excel file and a Compound Discoverer (CD) Results file. This module is dependant on Pandas. The Python code was written with Python 3.9.12.


tidyData() function

This function is expecting an Excel file with a "Compounds" sheet,
and a "Meta" sheet with the columns - 'Filename', 'SampleType', 'SampleID', 'Order', and 'Batch'.

This function will convert the Compounds table and Meta table into the TidyData format with a Data sheet and a Peak sheet.

In [ ]:
import CDExcelMessenger

# Set this variable to the path of your excel file
excelFilePath = "./testData.xlsx"

# Columns to keep and the names you would like to use for those columns
# If you put a '<' and '>' around a name, 
# this function will look for any column that starts with that name.
colsToKeepDict = {
    "Idx": "Idx",
    "UID": "UID",
    "Name": "Name",
    "Checked": "Checked",
    "Tags": "Tags",
    "Formula": "Formula",
    "RT [min]": "RT",
    "Calc. MW": "MW", 
    "MS2": "MS2",
    "# ChemSpider Results": "ChemSpiderRes",
    "# mzVault Results": "MzVaultRes",
    "# mzCloud Results": "MzCloudRes",
    "mzCloud Best Match": "mzCloudMatch",
    "mzVault Best Match": "mzVaultMatch",
    "<Mass List Match: >": "mzList_"
}

# Set "CIMCBlib" to True if you want to split peak names that start with ECU into two columns,
# Set "MSHit" to True if you want to get the sum of rows that have a MS2 hit 
# and to create the MS2Hit column.
# Set "mzmatch" to your chosen mzVault and mzCloud threshold
# Set "UIDPrefix" to the prefix you would like to have in the UID values
optionsDict = {
    "CIMCBlib": True,
    "MSHit": True,
    "mzmatch": 70,
    "UIDPrefix": "M"
}

CDExcelMessenger.tidyData(excelFilePath, colsToKeepDict, optionsDict)


updateCDResultsFile() function

updateCDResultsFile() is the function that allows you to import data from an Excel file into a CD Results file. This function can add new columns to the CD compound table, or update certain columns in the CD compound table (Tags, Checked, Name, or any columns already added by this function). A 'Cleaned' column will be added to the CD compound table that flags the rows in the Compound table that were found in the Excel file. An editible column called 'Notes' will be added to CD by default. A non-editible column called 'originalName' will be added to CD that contains the same data as the Name column. This will allow you to update the Name column while keeping the original Name data. This function also gives you the option of updating the Tag names and values in CD if you use 'tagList'. An MSI column will be added to the CD Results file based on whether or not certain Tags are checked.. If your Excel file doesn't have a 'compoundID' column, this function will add that column (make sure you don't use compoundID for something else). Keep 'compoundID' in the Excel file to improve performance the next time you run this function.



In [ ]:
import CDExcelMessenger

# Set these variables to the paths of the CD Results file and the Excel file.
cdResultsFilePath = "./testData.cdResult"
excelFilePath = "./testData.xlsx"

# Set this variable to the name of your Peak sheet in the Excel file
peakSheetName = "Peak"

# List of column names in the Excel file that you would like to update/add to the CD Results file.
excelColList = ["UID", "CIMCBlib", "Name", "qcRSD", "dRatio", "blankRatio", "Checked", "Notes"]

# If you include "Tags" in the tag list, 
# CD will contain the Tags found in the Tags Excel column (; should be the delimiter).
# You can use boolean/binary Excel columns as Tags if you inlcude the names of those columns.
# Any other names included in the tag list will still be added to the CD Tags.
# You can have up to 15 tags.
tagList = ["ms2Hit", "goodPeakShape", "goodRT", "queryMS", "msError", "mzVault", \
           "putativeCompound", "putativeClass"]

# Update the CD results file with Excel data.
CDExcelMessenger.updateCDResultsFile(
    cdResultsFilePath, 
    excelFilePath, 
    peakSheetName, 
    excelColList, 
    tagList
)

updateExcelFile() function

updateExcelFile() is the function that allows you to import data from a CD results file into an Excel file. This function doesn't allow you to export certain data from the CD compound table (e.g. Area). If you import the Tags data into the Excel file, this function will also add new columns with the data of the individual Tags, or update those columns if they already exist. An MSI column will be added to the Excel file based on whether or not certain Tags are checked. This function also gives you the option of removing rows that have been Checked in CD. If your Excel file doesn't have a 'compoundID' column, this function will add that column (make sure you don't use compoundID for something else). Keep 'compoundID' in the Excel file to improve performance the next time you run this function.

In [ ]:
import CDExcelMessenger

# Set these variables to the paths of the CD Results file and the Excel file.
cdResultsFilePath = "./testData.cdResult"
excelFilePath = "./testData.xlsx"

# Set these variables to the names of your Peak sheet and Data sheet in the Excel file
# Including the data sheet is optional
peakSheetName = "Peak"
dataSheetName = "Data"

# List of columns in the Excel file that you would like to update. 
excelColList = ["Tags", "Checked", "Notes"]

# Set this variable to True if you want to remove rows from the Excel file 
# that have been checked in CD (default is False)
removeCheckedRows = True

# Set this variable to True if you want to append the new sheets to the Excel file 
# (the sheet names will be given the suffix 'Appended'),
# If you set this variable to False, your Excel sheets will get written over (default is False)
appendSheets = True

# Update the Excel file with CD compound data 
CDExcelMessenger.updateExcelFile(
    cdResultsFilePath, 
    excelFilePath, 
    peakSheetName, 
    dataSheetName = dataSheetName,
    excelColList = excelColList,
    removeCheckedRows = removeCheckedRows,
    appendSheets = appendSheets
)


The following code will add an 'MSI' column to your Peak sheet in the Excel file. The MSI levels will be based on the tag columns that were added to the Excel file with the updateExcelFile() function.  

In [ ]:
# Import the Pandas module
import pandas as pd

# Set the path to the excel file and set the peak sheet name
excelFilePath = "./testData.xlsx"
peakSheetName = "PeakAppended"

# We're using a 'try' block to catch exceptions that can occur.
try:
    # import the peak data and store it in a dataframe
    peakTable = pd.read_excel(excelFilePath, sheet_name = peakSheetName)            
        
    # If the MSI column isn't in the peak table, add it
    if "MSI" not in peakTable.columns:
        # Add the MSI column with default values
        peakTable["MSI"] = ""

    # Get the number of rows in the peak table,
    # then loop through each row of the peak table
    peakRowCount = len(peakTable.index)
    for row in range(peakRowCount):
        
        # Get the values from the tag columns of the Excel file 
        # that will be used to determine the MSI level
        rtTag = peakTable.at[row,"goodRT"]
        mzVaultTag = peakTable.at[row,"mzVault"]
        compoundTag = peakTable.at[row,"putativeCompound"]
        classTag = peakTable.at[row,"putativeClass"]
    
        # Check if the current peak is MSI - level 0
        if rtTag and mzVaultTag:
            msi = "0"
            
        # Check if the current peak is MSI - level 1
        elif rtTag or mzVaultTag:
            msi = "1"

        # Check if the current peak is MSI - level 2
        elif compoundTag:
            msi = "2"
        
        # Check if the current peak is MSI - level 3
        elif classTag:
            msi = "3"
        
        # If the current peak didn't meet the requirements for any MSI levels
        else:
            msi = ""
            # Set the Checked value to True for the current peak
            peakTable.at[row,"Checked"] = True
    
        # Set the MSI value of the current peak
        peakTable.at[row,"MSI"] = msi

    # Order the Excel columns, 'leftCols' should be a list of the columns
    # you would like to be the left most columns in the Excel file
    leftCols = ["Idx", "compoundID", "UID", "Name", "Notes", "MSI"]                
    peakTable = peakTable[leftCols + [c for c in peakTable if c not in leftCols]]

    # Update the Excel file 
    with pd.ExcelWriter(
        excelFilePath,
        mode="a",
        engine="openpyxl",
        if_sheet_exists="replace",
    ) as writer:
        peakTable.to_excel(writer, sheet_name=peakSheetName, index=False) 
    print(excelFilePath+" updated")
            
# If the Excel file doesn't have the correct sheet
except ValueError:
    print("ValueError: Can't find "+peakSheetName+" in "+excelFilePath)
        
# If the Excel file can't be found
except FileNotFoundError:
    print("FileNotFoundError: Can't find "+excelFilePath)
    
# If permission to the Excel file was denied
except PermissionError:
    print("PermissionError: Couldn't gain permission to the Excel File. Make sure "+excelFilePath+" is not open in another program")  

# If a column can't be found in the peak table
except KeyError as k:
    print("KeyError: a column couldn't be found in the peak table - "+str(k))
    
# If another exception occured
except Exception as e:
    print(e)